## 04 - bölüm 4 - Hızlı Fourier Dönüşümü

In [1]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import time
from cmath import exp, pi

Hızlı Fourier Dönüşümü (FFT), önemsiz durum elde edilene kadar giriş dizisini yinelemeli olarak iki parçaya bölen bir *böl ve yönet* algoritmasıdır: biri tek endeksler için, diğeri çift endeksler için.

Karmaşık üstellerin (sinüs ve kosinüs toplamına ayrıştırılabilen) periyodik ve simetrik olduğunu ve bu özelliklerden FFT'nin tanımlandığını unutmamak önemlidir.

Özellikle, $e^{-j \frac{2\pi}{N} x u}$'dan sabit terimi izole ediyoruz ve onu bir değişken olarak tanımlıyoruz: $W = e^{-j \frac{2\ pi}{N}}$. Not $W$ sabittir çünkü zaman örneklemesine ($x$ tarafından kontrol edilir) veya frekanslara ($u$) bağlı değildir.

Örneğin, 4 gözlemli bir sinyal için, yani $N=4$:


In [2]:
N = 4
W = exp(-1j*(2*pi)/N)
print(W)

(6.123233995736766e-17-1j)


Bu değer $u$ veya $x$'a bağlı değildir. FFT'yi uygulamak için kullanacağımız iki özellik şunlardır:
1. u,x'deki periyodiklik: $W_N^{ux} = W_N^{u(N+x)} = W_N^{(u+N)x}$

2. karmaşık eşleniğin simetrisi: $W_N^{u(N-x)} = W_N^{-ux} = (W_N^{ux})^*$
   örneğin $x=N$, $W_N^{uN} = e^{-j2\pi u} =1$ için bunu görmek kolaydır
   
Şimdi algoritmanın **bölme** adımını tanımlıyoruz. Bu, dönüşümü $x$'in çift ve tek endekslerine ayrıştırarak yapılır. Karmaşık bir gösterimden kaçınmak için dönüşümü $W$ değişkeni cinsinden ifade edelim:

$F(u) = \sum_{x=0}^{N-1} f(x)W_N^{ux}$

Şimdi $2x$ çift endekslerini ve $2x+1$ tek endekslerini değerlendirme fonksiyonunu yazıyoruz:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)W_N^{u(2x)} +  \sum_{x =0}^{N/2-1} f(2x+1)W_N^{u(2x+1)}$

$2x$ $0,2,4,6$ dizisini oluştururken $2x+1$ $1,3,5,7$ dizisini istediğimiz gibi oluşturduğuna dikkat edin, dolayısıyla:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot(W_N^2)^{ux} +  \sum_{x =0}^{N/2-1} f(2x+1)\cdot (W_N^2)^{(2x+1)u}$

$x$'den bağımsız terimleri, yani $W_N^u$'ı ayırarak bu toplamı değiştirelim:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot(W_N^2)^{ux} +  W_N^u \cdot \sum_{x =0}^{N/2-1} f(2x+1)\cdot (W_N^2)^{ux}$

Ama biz öyle değiliz
$W_N^2 = e^{-j\frac{2\pi}{N}2} = e^{-j\frac{2\pi}{N/2}} = W_{N/2}$

ve bu 'numara'yı tanımlar, çünkü dönüşümü şu şekilde yazmaya izin verir:

$F(u) = \sum_{x = 0}^{N/2-1} f(2x)\cdot W_{N/2}^{ux} + W_N^u \cdot \sum_{x =0} ^{N/2-1} f(2x+1)\cdot W_{N/2}^{ux}$

İlk terim çift endekslere karşılık gelen $N/2$ öğelerinin DFT'sidir, ikinci terim ise tek endekslere ilişkin $N/2$ öğelerinin DFT'sidir.

Bu şekilde, $N$ öğelerinin DFT'sini yinelemeli bir şekilde iki $N/2$ DFT'ye bölebilir ve daha sonra sonuçları birleştirebiliriz:

$F(u) = F_\text{çift}(u) + W_N^u \cdot F_\text{tek}(u)$

Karmaşık eşleniklerin simetri özelliğini hatırlayın:

$F(u+N/2) = F_\text{çift}(u) - W_N^u \cdot F_\text{tek}(u)$

In [3]:
# küçük bir örnekle bu fikri kavramaya çalışalım
N = 4
f = [0,100,200,300]
f

[0, 100, 200, 300]

In [4]:
# diziyi çift ve tek indekslere bölme
f_even= f[0::2]
f_odd = f[1::2]
print(f_even)
print(f_odd)

[0, 200]
[100, 300]


In [5]:
# yinelemeli olarak, elde edilen dizileri, ilk olarak çift olanı, çift ve tek indekslere böleriz
f_even_even = f_even[0::2]
f_even_odd = f_even[1::2]
print(f_even_even)
print(f_even_odd)

[0]
[200]


Bu basit örnekte, temel duruma ulaşana kadar, yani yalnızca 1 çift ve 1 tek öğe olduğunda öğeleri bölümlendiriyoruz ve hesaplamaya izin veriyoruz:

In [6]:
reseven0 = f_even_even[0] + exp(-2j*pi*0/N) * f_even_odd[0]
reseven1 = f_even_even[0] - exp(-2j*pi*0/N) * f_even_odd[0]
reseven = [reseven0, reseven1]
print(reseven)

[(200+0j), (-200+0j)]


Şimdi bu sonuç saklanır ve ilk tek indekslere göre özyinelemenin diğer 'tarafını' yürütürüz

In [7]:
# tek endeksleri çift ve tek endekslere ayırın
f_odd_even = f_odd[0::2]
f_odd_odd = f_odd[1::2]
print(f_odd_even)
print(f_odd_odd)

[100]
[300]


In [8]:
resodd0 = f_odd_even[0] + exp(-2j*pi*0/N) * f_odd_odd[0]
resodd1 = f_odd_even[0] - exp(-2j*pi*0/N) * f_odd_odd[0]
resodd = [resodd0, resodd1]
print(resodd)

[(400+0j), (-200+0j)]


Şimdi bireysel sonuçları birleştirelim (yeniden eşitle ve yeniden sırala):

In [9]:
# simetrik özelliğinden 0 sonucunu şunun için de kullanabilirim:
# sinyali değiştirerek 0+N/2 = N/4 = 2 değerini elde edin
res0 = reseven[0] + exp(-2j*pi*0/N) * resodd[0]
res2 = reseven[0] - exp(-2j*pi*0/N) * resodd[0]

# benzer şekilde sonuç 1, 1+N/2 = 1+N/4 = 3 sonucunu elde etmek için kullanılır
res1 = reseven[1] + exp(-2j*pi*1/N) * resodd[1]
res3 = reseven[1] - exp(-2j*pi*1/N) * resodd[1]

#her şeyi bir araya getirmek
F_manual =  np.array([res0, res1, res2, res3]).astype(np.complex64)
print(F_manual)

[ 600.  +0.j -200.+200.j -200.  +0.j -200.-200.j]


Bu algoritma için bir fonksiyon kodlayalım

In [10]:
def FFT(f):
    N = len(f)
    if N <= 1:
        return f

    # bölüm
    even= FFT(f[0::2])
    odd = FFT(f[1::2])

    # sonuçların kombinasyonunu saklayın
    temp = np.zeros(N).astype(np.complex64)

    # yalnızca frekansların yarısını hesaplamak için gerekli
    # u+N/2 simetri özelliğinden elde edilebildiğinden
    for u in range(N//2):
        temp[u] = even[u] + exp(-2j*pi*u/N) * odd[u] # fethetmek
        temp[u+N//2] = even[u] - exp(-2j*pi*u/N)*odd[u]  # fethetmek

    return temp

In [11]:
# manuel hesaplamayla eşleşip eşleşmediğini görmek için fonksiyonun test edilmesi
F_fft = FFT(f)
print(F_fft)

[ 600.  +0.j -200.+200.j -200.  +0.j -200.-200.j]


In [12]:
# DFT1D ile karşılaştıralım
def DFT1D(f):
    # karmaşık katsayılardan oluşan boş bir dizi oluştur
    F = np.zeros(f.shape, dtype=np.complex64)
    n = f.shape[0]

    # x için indeksler oluşturarak çarpma işleminin numpy (f*exp) kullanılarak hesaplanmasına olanak sağlar
    x = np.arange(n)
    # her 'u' frekansı için vektörel çarpım ve toplam yapın
    for u in np.arange(n):
        F[u] = np.sum(f*np.exp( (-1j * 2 * np.pi * u*x) / n ))

    return F

F_dft = DFT1D(np.array(f))
print(F_dft)

[ 600.+0.000000e+00j -200.+2.000000e+02j -200.-7.347881e-14j
 -200.-2.000000e+02j]


In [13]:
# 3 sonucun yazdırılması
print(F_dft)
print(F_manual)
print(F_fft)

[ 600.+0.000000e+00j -200.+2.000000e+02j -200.-7.347881e-14j
 -200.-2.000000e+02j]
[ 600.  +0.j -200.+200.j -200.  +0.j -200.-200.j]
[ 600.  +0.j -200.+200.j -200.  +0.j -200.-200.j]


Yaklaşıklık nedeniyle bazen sonuçlar arasında küçük bir hata gözlenir.

DFT ve FFT'nin çalışma sürelerini karşılaştıralım:

In [14]:
# 10000 elemanlı bir dizi
t = np.arange(0, 2, 0.0001)
f = 1*np.sin(t*(2*np.pi) * 2) + 0.6*np.cos(t*(2*np.pi) * 8) + 0.4*np.cos(t*(2*np.pi) * 16)
print(t.shape)

(20000,)


In [15]:
start = time.time()
F_dft = DFT1D(np.array(f))
end = time.time()
elapsed = end - start
print("DFT Running time: " + str(elapsed) + " sec.")

start = time.time()
F_fft = FFT(np.array(f))
end = time.time()
elapsed = end - start
print("FFT Running time: " + str(elapsed) + " sec.")

DFT Running time: 29.59745502471924 sec.
FFT Running time: 1.5237364768981934 sec.


FFT'nin bu uygulamasının, girdinin $n = 2^p$ boyutunda olduğunu varsaydığını unutmayın; bu nedenle, $n$ ikinin katı olmadığında bu sorunun üstesinden gelmek için alternatifler kullanmak zorunda kalacağız (kullanımına izin veren algoritmalar vardır). herhangi bir $n$ değeri). Her durumda, $O(N \log N)$ karmaşıklığındaki Fourier Dönüşümünün hesaplanmasına izin veren bir algoritmaya değer.